In [1]:
import pandas as pd;
import numpy as  np;
from scipy.optimize import minimize;
import random;

In [2]:
ratings_large = pd.read_csv('ml-latest//ratings.csv')
#np.genfromtxt('ml-latest//ratings.csv',delimiter=',',skip_header =1)[:,0:3]

In [3]:
ratings_large.shape

(27753444, 4)

In [4]:
#users = ratings_large.
users = ratings_large.userId.unique()
movies = ratings_large.movieId.unique()

#(ratings_large[:,0])
#movies = np.unique(ratings_large[:,1])

In [16]:
#len(set(users_sub)),len(set(movies_sub)),len(set(users)),len(set(movies)),len(users),len(movies)
#N=9999
#users_sub = users[random.sample(list((range(0,283227))),N)]
#[[random.randint(0, 283227) for _ in range(N)]]

In [ ]:
np.random.seed(1234)

In [8]:
M=99
movies_sub = movies[random.sample(list((range(0,53889))),M)]

In [9]:
ratings_large.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [12]:
#ratings_sub = ratings_large[(ratings_large.userId.isin(users_sub))]
ratings_sub = ratings_large[(ratings_large.movieId.isin(movies_sub))]

In [17]:
ratings_sub.head()

,userId,movieId,rating,timestamp
454,4,3208,4.0,1113796776
567,4,4643,3.0,1117080240
762,4,43936,4.5,1145937382
895,7,1041,3.5,1238493451
1297,15,365,3.5,1370809370


In [177]:
#def create_subset(N,M,df):
#    i=N
#    while(1):
#        users = df.userId.unique()
#        users_sub = users[random.sample(list((range(0,len(users)))),i)]
#        ratings_sub = df[(df.userId.isin(users_sub))]
#        movies = ratings_sub.movieId.unique()
#        movies_sub = movies[random.sample(list((range(0,len(movies)))),M)]
#        ratings_sub = ratings_sub[(ratings_sub.movieId.isin(movies_sub))]
#        
#        i=i+int(N/10)
#        if(ratings_sub.userId.nunique()>N and ratings_sub.movieId.nunique()>M):
#            return ratings_sub
#        
#        if(i==len(users)):
#            return "Not possible"

In [18]:
np_ratings_sub = np.array(ratings_sub.pivot('userId','movieId',"rating"))

In [19]:
np_ratings_sub.shape

(37488, 99)

In [23]:
def get_rhat(u,v):
    return np.matmul(u,np.transpose(v))

In [24]:
#min J= 1/2||R - Utranspose(V)||**2
###Calculate reconstruction error

def cal_error(r,rhat):
    return 0.5 * np.nansum(np.square(r - rhat))    

In [25]:
def matrix_factorization(r,alpha,beta,steps,concept=3):
    #r = np.array(pd.DataFrame(ratings).pivot(0,1,2))  
    #r=ratings
    row = (r.shape[0])
    col = (r.shape[1])
    
    np.random.seed(1234)
    u = np.random.rand(row,concept)
    v = np.random.rand(col,concept)
    
    e = r - get_rhat(u,v)
    e = np.array(pd.DataFrame(e).fillna(0))
    #e1 = np.zeros((row,col))
    #
    #for i in range(row):
    #    for j in range(col):
    #        if r[i][j] > 0:
    #            e1[i,j] = r[i][j] - get_rhat(u,v)[i,j]
    #           
               
    #e = r - get_rhat(u,v)
    #print(e)
    #print(e1)
    for s in range(steps):
        u_c = u + alpha * (np.matmul(e,v) - beta * u)
        v_c = v + alpha * (np.matmul(e.T,u) - beta * v)
       
        u = np.copy(u_c)
        v = np.copy(v_c)
        e = r - get_rhat(u,v)
        np.nan_to_num(e,copy=False)
        #e = np.array(pd.DataFrame(e).fillna(0))
        if s%5 ==0:
            print(cal_error(r,get_rhat(u,v)))
    return get_rhat(u,v)

In [ ]:
matrix_factorization(np_ratings_sub,0.0002,0.02,10000)

244415.89331119927
3275607.480126619
58585.89835353549
54394.999677141976
50859.93583221495
47760.28322298443
45014.50587440171
42567.322083214785
40375.33218659358
38402.89348496418
36620.25583892906
35002.40484894336
33528.21119032043
32179.758627729723
30941.799312093728
29801.308578176395
28747.12005845081
27769.626099576544
26860.531255190814
26012.648860153495
25219.732567327086
24476.336297078768
23777.697339264538
23119.6383902042
22498.485140625242
21910.99669227166
21354.306604797854
20825.87278953388
20323.43479621796
19844.977301496943
19388.698818466728
18952.984816058626
18536.384574382908
18137.59121394631
17755.424428176528
17388.815523956153
17036.794437062697
16698.478441011026
16373.062310775586
16059.8097387943
15758.04583078176
15467.150534217237
15186.552873742872
14915.72588577422
14654.182159936883
14401.469907947063
14157.169491618693
13920.890351114636
13692.268282616687
13470.96302148185
13256.656092857931
13049.048896799924
12847.860999282662
12652.828604252

3656.7522479576073
3649.3258281545295
3641.8819603470897
3634.4208237860475
3626.9426011356704
3619.447478432757
3611.935645044837
3604.40729362762
3596.862620081718
3589.3018235085888
3581.7251061658026
3574.1326734215854
3566.5247337086876
3558.901498477604
3551.263182149131
3543.6100020663157
3535.9421784457854
3528.2599343285306
3520.563495530053
3512.853090590054
3505.1289507215447
3497.39130975947
3489.640404108855
3481.8764726924896
3474.0997568981647
3466.3105005255225
3458.5089497324625
3450.6953529812176
3442.869960984063
3435.0330266486612
3427.1848050231774
3419.3255532409958
3411.4555304652777
3403.574997833189
3395.684218399958
3387.783457082703
3379.87298060408
3371.9530574357636
3364.023957741812
3356.0859533218677
3348.139317554298
3340.1843253392185
3332.2212530414913
3324.2503784336277
3316.271980638731
3308.2863400733986
3300.2937383906033
3292.2944584226966
3284.288784124364
3276.2770005157267
3268.2593936254607
3260.236250434073
3252.2078588172485
3244.17450748936

1279.6736951110747
1277.735726392467
1275.8062931670158
1273.8853476833176
1271.97284254147
1270.0687306904176
1268.1729654252927
1266.285500384734
1264.406289548208
1262.5352872333199
1260.6724480931232
1258.8177271134118
1256.9710796100246
1255.132461226136
1253.3018279295447
1251.4791360099673
1249.6643420763246
1247.8574030540228
1246.0582761822716
1244.2669190113368
1242.4832893998687
1240.7073455121833
1238.9390458155697
1237.1783490775993
1235.4252143634342
1233.679601033148
1231.9414687390627
1230.2107774230624
1228.4874873139543
1226.7715589248132
1225.0629530503377
1223.3616307642296
1221.6675534165706
1219.9806826312097
1218.3009803031828
1216.6284085961138
1214.962929939651
1213.3045070269127
1211.6531028119307
1210.0086805071392
1208.3712035808433
1206.74063575473
1205.1169410013736
1203.5000835417807
1201.8900278429194
1200.2867386152968
1198.6901808105406
1197.100319618979
1195.5171204672813
1193.9405490160718
1192.3705711575838
1190.8071530133386
1189.2502609318258
1187

In [52]:
ratings_small.shape

#np.array(pd.DataFrame(ratings_small[1:5,:]).pivot(0,1,2)) 

(100836, 3)